In [11]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import random

In [12]:
transform = transforms.Compose([transforms.ToTensor()])

train_set = ImageFolder(root='cinic10/versions/1/train//', transform=transform)
validate_set = ImageFolder(root='cinic10/versions/1/valid//', transform=transform)
test_set = ImageFolder(root='cinic10/versions/1/test//', transform=transform)

class_names = train_set.classes
num_labels = len(class_names)

def prepare_transforms():
    
    train_set = ImageFolder(root='cinic10/versions/1/train//', transform=transform_default)
    validate_set = ImageFolder(root='cinic10/versions/1/valid//', transform=transform_default)
    test_set = ImageFolder(root='cinic10/versions/1/test//', transform=transform_default)
        
    data_loader = DataLoader(train_set, batch_size=64, num_workers=6, generator=torch.Generator(device='cpu'),pin_memory=True, shuffle=True,persistent_workers=True, prefetch_factor=4)
    data_loader_val = DataLoader(validate_set, batch_size=64, num_workers=3, generator=torch.Generator(device='cpu'),pin_memory=True, shuffle=True,persistent_workers=True, prefetch_factor=4)
    data_loader_test = DataLoader(test_set, batch_size=64, num_workers=3, generator=torch.Generator(device='cpu'),persistent_workers=True)

    return data_loader, data_loader_val, data_loader_test





In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_2_dropout(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(CNN_2_dropout, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=4, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(p=0.25)
        self.linear1 = nn.Linear(64 * 3 * 3  , 100)
        self.linear2 = nn.Linear(100, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = self.dropout(x)
        x = torch.flatten(x, 1)  # Startujemy od wymiaru 1, aby nie spłaszczać batch_size
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x


In [14]:
from torchvision.transforms import v2

cutmix = v2.CutMix(num_classes=num_labels)

def prepare_model_resnet_cutmix(lr):
    model = CNN_2_dropout()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    return model, optimizer ,criterion

In [15]:
def test(model,data_loader_test, criterion,device):
    test_loss = 0
    correct = 0
    total = 0
    for i, (batch_X, batch_Y) in enumerate(data_loader_test):
        X = batch_X.to(device, non_blocking=True)
        Y = batch_Y.to(device, non_blocking=True)
        outputs = model(X)
        loss = criterion(outputs, Y)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == Y).sum().item()
        total += Y.size(0)
    avg_test_loss = test_loss / len(data_loader_test)
    test_accuracy = correct / total * 100
    print(f" Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")            
    return  avg_test_loss, test_accuracy

In [18]:
def model_train_cutmix(lr, model, seed):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cutmix = v2.CutMix(num_classes=num_labels)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

    
    losses_tr = []
    accuracies_tr = []
    losses_val = []
    accuracies_val = []
    data_loader, data_loader_val, data_loader_test = prepare_transforms()
            
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
        
    num_epochs = 60
    
    prev_prev_prev_loss = float('inf')
    prev_prev_loss = float('inf')
    prev_loss = float('inf')
    curr_loss = float('inf')

    model = model()
    model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(num_epochs): 
        print('Entered the loop')
        model.train()
        total_loss = 0
        correct = 0
        total = 0
                
        for i, (batch_X, batch_Y) in enumerate(data_loader):
            batch_X, batch_Y = cutmix(batch_X, batch_Y)
            X = batch_X.to(device, non_blocking=True)  
            Y = batch_Y.to(device, non_blocking=True)  
            optimizer.zero_grad()
            outputs = model(X)
            loss = torch.nn.functional.cross_entropy(outputs,Y)
            loss.backward()
            optimizer.step()
                
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
                
    
        avg_loss = total_loss / len(data_loader)
        train_losses.append(avg_loss)
        print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")
            
            
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for i, (batch_X, batch_Y) in enumerate(data_loader_val):
                if i >= 140:  
                    break
                X = batch_X.to(device, non_blocking=True)
                Y = batch_Y.to(device, non_blocking=True)
                outputs = model(X)
                loss = criterion(outputs, Y)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == Y).sum().item()
                total += Y.size(0)
        avg_val_loss = val_loss / (140)
        val_accuracy = correct / total * 100
        print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")            
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_accuracy)
    
        if(avg_val_loss>curr_loss and curr_loss>prev_loss and prev_loss > prev_prev_loss and prev_prev_loss> prev_prev_prev_loss):
            losses_tr.append(train_losses)
            accuracies_tr.append(train_accuracies) 
            losses_val.append(val_losses)  
            accuracies_val.append(val_accuracies)
            avg_test_loss, test_accuracy = test(model, data_loader_test, criterion, device)
            break
        prev_prev_prev_loss = prev_prev_loss  
        prev_prev_loss = prev_loss
        prev_loss = curr_loss
        curr_loss = avg_val_loss
    
        if(epoch == num_epochs - 1 ):
            losses_tr.append(train_losses)
            accuracies_tr.append(train_accuracies) 
            losses_val.append(val_losses)  
            accuracies_val.append(val_accuracies) 
            avg_test_loss, test_accuracy = test(model, data_loader_test, criterion, device)

    return model,lr,val_losses, val_accuracies, train_losses, avg_test_loss, test_accuracy




In [19]:
lrates = [0.001,0.0001]
results = []
for lr in lrates:
    results.append(model_train_cutmix(lr,CNN_2_dropout,42))

Entered the loop
Epoch 1, Average Loss: 2.0562
Epoch 1, Validation Loss: 1.6784, Accuracy: 39.07%
Entered the loop
Epoch 2, Average Loss: 1.9551
Epoch 2, Validation Loss: 1.5940, Accuracy: 42.04%
Entered the loop
Epoch 3, Average Loss: 1.9065
Epoch 3, Validation Loss: 1.5309, Accuracy: 44.45%
Entered the loop
Epoch 4, Average Loss: 1.8731
Epoch 4, Validation Loss: 1.4815, Accuracy: 46.13%
Entered the loop
Epoch 5, Average Loss: 1.8421
Epoch 5, Validation Loss: 1.4077, Accuracy: 51.62%
Entered the loop
Epoch 6, Average Loss: 1.8217
Epoch 6, Validation Loss: 1.3508, Accuracy: 53.11%
Entered the loop
Epoch 7, Average Loss: 1.8062
Epoch 7, Validation Loss: 1.3476, Accuracy: 53.50%
Entered the loop
Epoch 8, Average Loss: 1.7956
Epoch 8, Validation Loss: 1.3787, Accuracy: 52.33%
Entered the loop
Epoch 9, Average Loss: 1.7815
Epoch 9, Validation Loss: 1.3139, Accuracy: 53.64%
Entered the loop
Epoch 10, Average Loss: 1.7815
Epoch 10, Validation Loss: 1.2910, Accuracy: 55.30%
Entered the loop
E

In [20]:
results

[(CNN_2_dropout(
    (conv1): Conv2d(3, 16, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (dropout): Dropout(p=0.25, inplace=False)
    (linear1): Linear(in_features=576, out_features=100, bias=True)
    (linear2): Linear(in_features=100, out_features=10, bias=True)
  ),
  0.001,
  [1.6784299084118435,
   1.5940250354153769,
   1.530903683389936,
   1.4815372722489493,
   1.4076869479247502,
   1.35075124672481,
   1.34758323601314,
   1.3786517117704664,
   1.3139239404882703,
   1.2910399300711495,
   1.315556012732642,
   1.2550739169120788,
   1.2481779562575